<a href="https://colab.research.google.com/github/univer-coder/Deep-learning-course/blob/main/lecture02_homework.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 第2回講義 宿題

## 課題
今回のLessonで学んだことを元に，MNISTのファッション版 (Fashion MNIST，クラス数10) をソフトマックス回帰によって分類してみましょう．

Fashion MNISTの詳細については以下のリンクを参考にしてください．

Fashion MNIST: https://github.com/zalandoresearch/fashion-mnist

### 目標値
Accuracy: 80%

### ルール
- 訓練データは`x_train`， `y_train`，テストデータは`x_test`で与えられます．
- 予測ラベルは one_hot表現ではなく0~9のクラスラベル で表してください．
- **下のセルで指定されている`x_train、y_train`以外の学習データは使わないでください．**
- **ソフトマックス回帰のアルゴリズム部分の実装はnumpyのみで行ってください** (sklearnやtensorflowなどは使用しないでください)．
    - データの前処理部分でsklearnの関数を使う (例えば `sklearn.model_selection.train_test_split`) のは問題ありません．

### 提出方法
- 2つのファイルを提出していただきます．
    1. テストデータ (`x_test`) に対する予測ラベルを`submission_pred.csv`として保存し，**Omnicampusの宿題タブから「第2回 機械学習基礎」を選択して**提出してください．
    2. それに対応するpythonのコードを`submission_code.py`として保存し，**Omnicampusの宿題タブから「第2回 機械学習基礎 (code)」を選択して**提出してください．pythonファイル自体の提出ではなく，「提出内容」の部分にコードをコピー&ペーストしてください．
      
- なお，採点は1で行い，2はコードの確認用として利用します（成績優秀者はコード内容を公開させていただくかもしれません）．コードの内容を変更した場合は，**1と2の両方を提出し直してください**．

### 評価方法
- 予測ラベルの`y_test`に対する精度 (Accuracy) で評価します．
- 即時採点しLeader Boardを更新します（採点スケジュールは別アナウンス）．
- 締切時の点数を最終的な評価とします．

### ドライブのマウント

In [32]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### データの読み込み（このセルは修正しないでください）

In [33]:
import os
import sys

import numpy as np
import pandas as pd

sys.modules['tensorflow'] = None

def load_fashionmnist():
    # 学習データ
    x_train = np.load('drive/MyDrive/Colab Notebooks/DLBasics2025_colab/Lecture02/data/x_train.npy')
    y_train = np.load('drive/MyDrive/Colab Notebooks/DLBasics2025_colab/Lecture02/data/y_train.npy')

    # テストデータ
    x_test = np.load('drive/MyDrive/Colab Notebooks/DLBasics2025_colab/Lecture02/data/x_test.npy')

    x_train = x_train.reshape(-1, 784).astype('float32') / 255
    y_train = np.eye(10)[y_train.astype('int32')]
    x_test = x_test.reshape(-1, 784).astype('float32') / 255

    return x_train, y_train, x_test

In [37]:
%cd /content/

/content


### ソフトマックス回帰の実装

In [57]:
x_train, y_train, x_test = load_fashionmnist()

from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

def np_log(x):
    return np.log(np.clip(a=x, a_min=1e-10, a_max=1e+10))

def softmax(x, axis=-1):
    x -= x.max(axis, keepdims=True)
    x_exp =  np.exp(x)
    return  x_exp / x_exp.sum(axis, keepdims=True)

# ミニバッチ学習にして高速に学習させる
def get_minibatches(x, y, batch_size):
    indices = np.random.permutation(len(x))
    for i in range(0, len(x), batch_size):
        idx = indices[i:i+batch_size]
        yield x[idx], y[idx]

# 重み初期化（He初期化）
W1 = np.random.randn(784, 256).astype('float32') * np.sqrt(2 / 784)
b1 = np.zeros((256,), dtype='float32')
W2 = np.random.randn(256, 10).astype('float32') * np.sqrt(2 / 256)
b2 = np.zeros((10,), dtype='float32')

# 過学習を防ぐdropout
def dropout(x, drop_rate=0.3):
    mask = np.random.binomial(1, 1 - drop_rate, size=x.shape)
    return x * mask / (1 - drop_rate)

# 学習データと検証データに分割
x_train, x_valid, y_train, y_valid = train_test_split(x_train, y_train, test_size=0.1)

def train(x, t, eps=0.1):
    global W1, b1, W2, b2

    batch_size = x.shape[0]

    # 予測
    z1 = x @ W1 + b1
    h = np.maximum(0, z1) # 中間層ReLU関数を追加
    h = dropout(h, drop_rate=0.3)  # dropout
    y_hat = softmax(h @ W2 + b2) # 出力層Softmax関数

    # 目的関数の評価
    cost = (- t * np_log(y_hat)).sum(axis=1).mean()

    # delta2: 出力層の勾配∂E/∂z2
    delta2 = y_hat - t  # (batch_size, 出力の次元数)
    # z1: 中間層の線形出力 (x @ W1 + b1)
    relu_mask = (z1 > 0).astype(float)  # ReLUの勾配（0か1）
    # delta2 は softmax + cross entropy の勾配（y_hat - t）
    delta1 = delta2 @ W2.T * relu_mask  # 要素ごとの積でReLUを通す # ∂E/∂z1 = ∂E/∂h * ∂h/∂(z1)

    # パラメータの更新
    dW1 = x.T @ delta1 / batch_size # (入力の次元数, 出力の次元数)
    db1 = delta1.mean(axis=0)  # (出力の次元数,)
    dW2 = h.T @ delta2 / batch_size  # (入力の次元数, 出力の次元数)
    db2 = delta2.mean(axis=0)  # (出力の次元数,)\

    W1 -= eps * dW1
    b1 -= eps * db1
    W2 -= eps * dW2
    b2 -= eps * db2


    return cost

def valid(x, t):
    z1 = x @ W1 + b1
    h = np.maximum(0, z1) # 中間層ReLU関数を追加
    y_hat = softmax(h @ W2 + b2) # 出力層Softmax関数
    cost = (- t * np_log(y_hat)).sum(axis=1).mean()

    return cost, y_hat

for epoch in range(100):
    lr = 0.1 * (0.95 ** (epoch // 10))
    for xb, yb in get_minibatches(x_train, y_train, batch_size=128):
        train(xb, yb, eps=lr)

    cost, y_pred = valid(x_valid, y_valid)
    if epoch % 10 == 9 or epoch == 0:
        print('EPOCH: {}, Valid Cost: {:.3f}, Valid Accuracy: {:.3f}'.format(
                epoch + 1,
                cost,
                accuracy_score(y_valid.argmax(axis=1), y_pred.argmax(axis=1))
            ))

submission = pd.Series(y_pred.argmax(axis=1), name='label')
submission.to_csv('drive/MyDrive/松尾研AI講座/DL基礎講座/第2回課題/submission_pred.csv', header=True, index_label='id')

EPOCH: 1, Valid Cost: 0.681, Valid Accuracy: 0.734
EPOCH: 10, Valid Cost: 0.411, Valid Accuracy: 0.852
EPOCH: 20, Valid Cost: 0.408, Valid Accuracy: 0.855
EPOCH: 30, Valid Cost: 0.405, Valid Accuracy: 0.851


KeyboardInterrupt: 